In [1]:
#IMPORTING PACKAGES
from pixiedust.display.app import *
import pixiedust
import pandas as pd
import bokeh
import seaborn
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np
import requests
import datetime
import csv
import os

Pixiedust database opened successfully


In [4]:
#HELPER FUNCTIONS

def getData(response):
    response = json.loads(response.text)
    for record in response['events']:
        record['timestamp'] = pd.datetime.strptime(record['timestamp'][:-4], '%a, %d %b %Y %H:%M:%S')
        with open('data.csv', 'a') as f: 
            w = csv.DictWriter(f, record.keys())
            w.writerow(record)
    data = pd.read_csv('data.csv')
   
    column_names =  ['CPU','Instances', 'Timestamp']
    data.columns = column_names
    is_1 = data['CPU']==1
    data = data[is_1]
    data = data.drop(['CPU'], axis=1)
    data = data.sort_values(by=['Timestamp'])
    data = data.drop_duplicates()
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y-%m-%d %H:%M:%S')
    #TODO: Averaging
    data = data.set_index('Timestamp')
    data = data.resample("10T").sum()
    data = data.reset_index()
    return data


def parser(x):
    return pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

def aggregate(df, column_names,  period):
    df.columns = column_names
    print(df)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
    if 'CPU' in df.columns:
        is_1 = df['CPU']==1
        df = df[is_1]
        df = df.drop(['CPU'], axis=1)
    df = df.sort_values(by=['Timestamp'])
    print(df)
     
    df = df.set_index('Timestamp')
    df = df.resample(period).min()
    df = df.reset_index()
    return df

def processCSV(series):
#     series['Timestamp'] = series['Timestamp'].to_pytimestamp()
    series['Timestamp'] = pd.to_datetime(series['Timestamp'], format='%Y-%m-%d %H:%M')
    min_val=pd.to_datetime('2019-02-18 00:00')
    delta = np.timedelta64(1,'D')
    series['date'] = (series['Timestamp'] - min_val) / delta
    return series



In [34]:
#Training API

params = {'region':'London'}
response = requests.post('https://capacityoptimizer-appreciative-klipspringer.eu-gb.mybluemix.net/train',params = params)
print(response.text)

{"Adaboost Regression":-0.343850337565209,"Best Model":"Adaboost Regression","Best Score":-0.343850337565209,"Decision Tree Regression":-0.5553200263533056,"KNN Regression":-0.41505578837983864}



In [50]:
params = {'Region':'Tokyo','Days':'7'}
response = requests.get('https://capacityoptimizer-appreciative-klipspringer.eu-gb.mybluemix.net/fetch',params = params)
df = getData(response)
df = processCSV(df)


{'events': [{'instances': '119', 'cpuRequestPerPod': '1', 'timestamp': 'Fri, 03 May 2019 08:00:05 GMT'}, {'instances': '37', 'cpuRequestPerPod': '3', 'timestamp': 'Fri, 03 May 2019 08:00:07 GMT'}, {'instances': '57', 'cpuRequestPerPod': '2', 'timestamp': 'Fri, 03 May 2019 08:00:09 GMT'}, {'instances': '127', 'cpuRequestPerPod': '1', 'timestamp': 'Fri, 03 May 2019 08:10:10 GMT'}, {'instances': '40', 'cpuRequestPerPod': '3', 'timestamp': 'Fri, 03 May 2019 08:10:11 GMT'}, {'instances': '61', 'cpuRequestPerPod': '2', 'timestamp': 'Fri, 03 May 2019 08:10:13 GMT'}, {'instances': '35', 'cpuRequestPerPod': '3', 'timestamp': 'Fri, 03 May 2019 08:20:04 GMT'}, {'instances': '54', 'cpuRequestPerPod': '2', 'timestamp': 'Fri, 03 May 2019 08:20:06 GMT'}, {'instances': '115', 'cpuRequestPerPod': '1', 'timestamp': 'Fri, 03 May 2019 08:20:08 GMT'}, {'instances': '60', 'cpuRequestPerPod': '2', 'timestamp': 'Fri, 03 May 2019 08:30:09 GMT'}, {'instances': '41', 'cpuRequestPerPod': '3', 'timestamp': 'Fri, 0

In [27]:
labels = df['Timestamp']
values = df['Instances']



adaboost = []
decision = []
knn = []
for value in df['date']:
    params = {'X':value, 'Region':'London'}
    response = requests.get('https://capacityoptimizer-appreciative-klipspringer.eu-gb.mybluemix.net/predict',params = params)
    print(response.text)
    response = json.loads(response.text)
  
    adaboost.append(float(response["Adaboost Regression"]))
    decision.append(float(response["Decision Tree Regression"]))
    knn.append(float(response["KNN Regression"]))
    
# print(values2)

# params = {'X':max(series['date'])+1}
# response = requests.get('https://predictinstances-balanced-gerenuk.eu-gb.mybluemix.net/predict',params = params)
# response = json.loads(response.text)
# values2.append(response["y"])

 

# print(values2)
# graph_df = pd.DataFrame({'Timestamp': df,'Actual': values1,'Predicted': values2})

{"Adaboost Regression":"531.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"409.0"}

{"Adaboost Regression":"531.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"531.0"}

{"Adaboost Regression":"514.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"514.0"}

{"Adaboost Regression":"514.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"503.0"}

{"Adaboost Regression":"484.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"518.0"}

{"Adaboost Regression":"484.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"484.0"}

{"Adaboost Regression":"479.0","Best Model":"Adaboost Regression","Decision Tree Regression":"538.3230769230769","KNN Regression":"475.0"}

{"Adaboost Regressio

In [44]:
graph_df = pd.DataFrame({'Timestamp': labels,'Instances': values})
adaboost_df = pd.DataFrame({'Timestamp': labels,'Instances': adaboost})
knn_df = pd.DataFrame({'Timestamp': labels,'Instances': knn})
decision_df = pd.DataFrame({'Timestamp': labels,'Instances': decision})


In [45]:
display(graph_df)

In [43]:
new_df= pd.DataFrame({'Timestamp': labels,'Instances': values})
hour_df = aggregate(new_df, ['Instances','Timestamp'],  "H")
print(hour_df)
display(hour_df)

In [47]:
new_df= pd.DataFrame({'Timestamp': labels,'Instances': values})
month_df = aggregate(new_df, ['Instances','Timestamp'],  "M")
print(month_df)

In [31]:
display(adaboost_df)

In [32]:
display(knn_df)

In [33]:
display(decision_df)